# NOTE

- Make a copy of this template before you start editing, and exported the file as a ipynb where you are done.

- Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Runtime$\rightarrow$Factory reset runtime) and then **run all cells** (in the menubar, select Runtime$\rightarrow$Run all).

- Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Fill out your name below in the `NAME` variable, leave the `COLLABORATORS` variable empty.

In [1]:
NAME = "payaljm2"
COLLABORATORS = ""

---

# Assignment 3 - Phylogeny

This assignment will focus on phylogeny, particularly the reconstruction of phylogenetic trees given sequence information. There are 3 parts to this assignment. They will total to 50 points for this homework.

In [2]:
# Setup dependencies

! pip install nose

import nose.tools as nt
import copy
from itertools import combinations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 10.5 MB/s 


## Part 1 - The Four Point Condition - 10 points

We know that a given square distance matrix is additive if and only if the four point condition holds for every leaf quartet $i,j,k,$ and $l$. For your convenience the Four Point condition is shown below. $$d_{i,j} + d_{k,l} \leq d_{i,k} + d_{j,l} = d_{i,l} + d_{j,k}$$

Complete the function `is_additive`, that takes an $n\times n$ matrix $D$ and returns `True` if and only if D is additive.

In [3]:
def is_additive(D):
    """
    Returns true if the square matrix D is additive
    
    :param: D is an nxn list of lists of ints 
    :return: true if D is an additive distance matrix
    """
    n = len(D)
    # fourPointCombination= list(combinations(range(n), 4))
    # print(fourPointCombination)

    for i in range(n):
      for j in range(i+1, n):
        for k in range(j+1, n):
          for l in range(k+1, n):
            sum1 = D[i][j]+D[k][l]
            sum2 = D[i][k]+D[j][l]
            sum3 = D[i][l]+D[j][k]
            # if(sum1==sum2 and sum1<sum3)
              

            l =[sum1, sum2,sum3]
            maxsum = max(sum1, sum2, sum3)
            isfollowingRule = l.count(maxsum)>=2
            if isfollowingRule is False:
               return False;
    return True

In [4]:
### Student tests (You may add more)

D0 = [
    [0, 6, 13, 14],
    [6, 0, 15, 16],
    [13, 15, 0, 5],
    [14, 16, 5, 0]
]
D1 = [
    [0, 2, 4, 4],
    [2, 0, 4, 4],
    [4, 4, 0, 2],
    [4, 4, 2, 0]
]

nt.assert_true(is_additive(D0))

In [5]:
# HIDDEN TEST CELL DO NOT MODIFY


In [6]:
# HIDDEN TEST CELL DO NOT MODIFY


## Part 2 - Neighbor Joining - 20 points

Recall that the neighbor joining algorithm is an approach to solving the Large Additive Distance phylogeny problem:

Given $n\times n$ matrix $D = [d_{ij}]$, find tree $T$ with $n$ leaves and edge weights such that $\underset{(i,j)\in[n]^2}{max}\lvert d_T(i,j) - d_{i,j}\rvert$.

This effectively finds the closest $D'$ to the input matrix $D$ that is additive.

Your task here will be to complete the function `neighbor_join` which will take an input distance matrix $D$, and construct its phylogenetic tree using the neighbor joining algorithm.

The input `D` will be a list of lists. `D` will be defined on `n` leaves labelled with the integers `[0,..,n-1]` such that `D[i][j]` will be the distance between leaf `i` and leaf `j` $(i,j) \in [n]^2$. Your function should create `n-2` additional internal vertices`[n,...,2n-3]` (for a total of $n+(n-2) = 2n-2$ vertices) corresponding to the joined leaves, and other joined clusters. The function should output an edge list `T` which will be a dict of dicts. The existence of `T[p][q]` will represent an edge in the final tree where $(p,q) \in [2n-2]^2$, and the value of `T[p][q]` should be equal to the weight of the edge between node `p` and node `q`. Note that `T` does not have to contain a value for all `p,q`, however 

* If `T[p][q]` exists then its value must be the distance between the node `p` and the node `q`, and represents a single edge (not a path) between them.
* If `T[p][q]` exists, then `T[q][p]` must also exist and it must be the case that `T[p][q] == T[q][p]`

Note: The above rules mean that `T` must contain `2n-3` unique edges, or that it must have `4n-6` values.

**The neighbor joining algorithm is specified here for your convenience:**
Let `D` be an `mxm` matrix:
* Until you have only 2 clusters, or `len(D)==2` (because NJ constructs unrooted binary trees)
    * Compute `u_k = sum(D[k])` for all `0<=k<len(D)`
    * Find pair of nodes `(i,j)` that minimizes `S[i][j] = (m-2)*D[i][j] - u_i -u_j`. A helper function has been given to you for this.
    * Create a node for `[ij]` using the next unused integer in `[0,...,2n-3]`. Call it `r`.
    * Add edges `T[i][r], T[r][i], T[j][r],` and `T[r][j]`. The distance between the leaves and their corresponding internal node is: 
        * `T[r][i] = 0.5*(D[i][j] + (u_i - u_j)/(len(D)-2))`
        * `T[r][j] = 0.5*(D[i][j] + (u_j - u_i)/(len(D)-2))`
    * Precompute and add new row and column in `D` for `r` with distances:
        * `D[r][m] = 0.5*(D[i][m] + D[j][m] - D[i][j])` for all `m!=i` and `m!=j`.
    * Now we must remove the rows and columns for `i` and `j` from `D`.
    * Update your internal node counter.
* There is one last edge you may need to add here (why?).
    
*Testing your neighbor joining function is difficult, so we have provided a testing harness that takes an input matrix, and your output tree, and sums the paths inside the tree to recover a distance matrix between the leaves. The function `check_tree` will then compare the two matrices to see if all values are equal. Obviously, if your input matrix is not additive, this will not work. The hidden tests will test ONLY additive inputs. If you are writing your own test cases, we suggest you start with a tree, compute a distance matrix, then use it to check your code.*

In [7]:
import numpy as np

In [8]:
def check_tree(D,T):
    """
    Implement floyd-warshall algorithm on the graph defined in T
    
    """
    
    V = len(T)
    dT = [[float("inf") for i in range(V)] for j in range(V)]
    n = (V + 2)//2
    
    # Length check
    if n!=len(D):
        return False
    
    # fill in existing edges
    for k in T:
        for l in T[k]:
            dT[k][l] = T[k][l]
            dT[l][k] = T[l][k]
    
    # fill in the diagonal elements 
    for i in range(len(dT)):
        dT[i][i] = 0.0
    
    # relax edges 
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if dT[i][j] > dT[i][k] + dT[k][j]:
                    dT[i][j] = dT[i][k] + dT[k][j]
                    dT[j][i] = dT[i][j]
    
    for i in range(n) :
        print(dT[i][:n])
    # Check each value in dT
    for i in range(n):
        for j in range(n):
            if D[i][j] != dT[i][j]:
                return False
    return True

def min_S_value(D, u):
    """
    returns the value (i,j) for which 
    (m-2)*D[i][j] - u_i - u_j is minimum
    """
    m = len(D)
    min_S, min_i, min_j = float("inf"),-1,-1
    for k in D:
        for l in D[k]:
            if l!=k:
                crit = (m-2)*D[k][l] - u[k] - u[l]
                if crit < min_S:
                    min_S = crit
                    min_i = k
                    min_j = l
    return (min_i, min_j)


def neighbor_join(D):
    """
    Takes a distance matrix D, and returns the tree T 
    consistent with the closest additive matrix D' to D.
    
    :param: D is a dict of dicts representing pairwise distances between leaves
    :return: a dict of dicts that contains all the edges with their weights in the tree defined by D'.    
    """
    
    T = {}
    r = len(D)
    while len(D)>2:
        u={}
        # print("D1",D)
        for key in D:
          u[key]=sum(D[key].values())
        
        i, j = min_S_value(D, u)
        
        if(not T.__contains__(i)):
          T[i]={}
        if(not T.__contains__(r)):
          T[r]={}
        if(not T.__contains__(j)):
          T[j]={}
        T[i][r] = 0.5*(D[i][j] + (u[i] - u[j])/(len(D)-2))
        T[r][i] = 0.5*(D[i][j] + (u[i] - u[j])/(len(D)-2))
        T[r][j] = 0.5*(D[i][j] + (u[j] - u[i])/(len(D)-2))
        T[j][r] = 0.5*(D[i][j] + (u[j] - u[i])/(len(D)-2))
        
        D[r]={}
        
        for m in D:
          
          if m!=i and m!=j and m!=r:
            D[r][m] = 0.5*(D[i][m] + D[j][m] - D[i][j]) #for all m!=i and m!=j.
        D[r][r] = 0
        del D[i]
        del D[j]
        for key in D:
          D[key][r] = D[r][key]
          if(D[key].__contains__(i)):
            del D[key][i]
          if(D[key].__contains__(j)):
            del D[key][j]
        r=r+1
    # print("T",T)
    # print("D",D)
    if len(D)==2:
       keys = list(D.keys())
       i, j = keys[0], keys[1]
       if(not T.__contains__(i)):
          T[i]={}
       if(not T.__contains__(j)):
          T[j]={}
       if(D[i].__contains__(j)):
        T[i][j] = D[i][j]
        T[j][i] = D[i][j]
       elif D[j].__contains__(i):
         T[i][j] = D[j][i]
         T[j][i] = D[j][i]
       

    return T

In [9]:
### Student tests (You may add more)
D0 = {
    0: {0:0,1:2,2:4},
    1: {0:2,1:0,2:2},
    2: {0:4,1:2,2:0}
}

D1 = {
    0: {0:0,1:2,2:4,3:4},
    1: {0:2,1:0,2:4,3:4},
    2: {0:4,1:4,2:0,3:2},
    3: {0:4,1:4,2:2,3:0}
}



# D2 = {
#     0: {0:0,1:5,2:4,3:7, 4:6, 5:8},
#     1: {0:5,1:0,2:7,3:10,4:9, 5:7},
#     2: {0:4,1:7,2:0,3:7, 4:6, 5:8},
#     3: {0:7,1:10,2:7,3:0, 4:5, 5:9},
#     4: {0:6,1:9,2:6,3:5, 4:0, 5:8},
#     5: {0:8,1:7,2:8,3:9, 4:8, 5:0}
# }
        
nt.assert_true(check_tree(copy.deepcopy(D0), neighbor_join(D0)))        
#nt.assert_true(check_tree(copy.deepcopy(D1), neighbor_join(D1)))
neighbor_join(D1)
# print("D2")
# neighbor_join(D2)

[0.0, 2.0, 4.0]
[2.0, 0.0, 2.0]
[4.0, 2.0, 0.0]


{0: {4: 1.0},
 4: {0: 1.0, 1: 1.0, 5: 2.0},
 1: {4: 1.0},
 2: {5: 1.0},
 5: {2: 1.0, 3: 1.0, 4: 2.0},
 3: {5: 1.0}}

In [10]:
# HIDDEN TEST CELL DO NOT MODIFY


In [11]:
# HIDDEN TEST CELL DO NOT MODIFY


## Part 3 - Small Maximum Parsimony Phylogeny Problem - 20 points

Given an $m\times n$ matrix $A = [a_{ij}]$ and a tree $T$ with $m$ leaves, find the assignment of character states to the internal vertices of the tree that results in the minimum parsimony score.

This problem is known as the small maximum parsimony phylogeny problem. This differs from the large problem in that you are given a phylogenetic tree.

The algorithm you will implement is the Sankoff Algorithm shown in detail here https://cs.brown.edu/courses/csci1950-z/slides/CSCI1950ZFall09_Lecture2.pdf. The recurrence for the algorithm is given here for your convenience.

Given rooted tree $T$ whose leaves are labelled by $\sigma:L(T)\rightarrow\Sigma$

$$c(s,t) = \begin{cases}
0 & \text{if $s=t$}\\
1,& \text{if $s\neq t$}
\end{cases}$$

$$\mu(v,s) = min\begin{cases}
\infty & \text{if $v \in L(T)$ and $s\neq \sigma(v)$}\\
0,& \text{if $v \in L(T)$ and $s=\sigma(v)$}\\
\sum_{w\in\delta(v)}\min_{t\in\Sigma}\{c(s,t) + \mu(s,t)\},& \text{if $v\notin L(T)$}\\
\end{cases}$$

Here, $\mu(v,s)$ is the minimum number of mutations in the subtree rooted at $v$ when assigning state $s$ to $v$, and $\delta(v)$ is the set of children of $v$.


Given to you is the definition `Node` of a single vertex in the tree. Most of the information is stored in the nodes themselves.
* `Node.name` is just a way to label the nodes to make it easier to understand the structure of the tree. 
* `Node.state` is the character assigned to that node
* `Node.leaf` is a bool indicating  whether the node is a leaf
* `Node.root` is a bool indicating whether the node is the root of the tree
* `Node.parent` is a reference to the parent of the current node. This will be useful for backtraces.
* `Node.children` is a list of references to the children nodes of the current node
* `Node.mu` is a dict that maps the possible state assignments of the node to the minimum number of mutations in the subtree rooted at this node. You will primarily be modifying this data structure.
* `Node.min_val` stores the minimum `mu` value for that node. ** You must at least set this for the root of your tree.**

Complete the function `sankoff_fill` which takes a reference to the root_node of a tree that has the states filled in for its leaves, as well as a state_alphabet (a list of all possible character states). This function does not return anything, however it modifies the `Node.mu` for all of the nodes in the tree rooted at `root_node`. You must use the recurrence shown above.

Also complete the `sankoff_backtrace` function which takes a reference to the root of a filled in tree `root_node`, as well as all possible characters as a list `state alphabet`, and performs a backtrace in order to fill in the states of each node in the tree with the correct character from the state alphabet. 

**NOTE**: There may be more than one solution that gives you the same parsimony score. You should break ties in the order they appear in the `state alphabet`.

In [12]:
import math

In [13]:
class Node:
    def __init__(self, name,  character, states_alphabet, root=False, leaf=False):
        self.name = name
        self.state = character
        self.leaf = leaf
        self.root = root
        self.parent = None
        self.children = []
        self.mu = {s:float("inf") for s in states_alphabet}
        self.min_val = None
        
    def add_child(self, child):
        self.children.append(child)
        child.assign_parent(self)
        
    def assign_parent(self, parent):
        self.parent = parent

        
def cost(s,t):
    """
    Computes the cost of a mutation between s and t
    """
    return 0 if s==t else 1


def sankoff_fill(root_node, states_alphabet):
    """
    Takes a reference to the root of a character based phylogeny tree,
    and fills in the mu values for all nodes
    
    """
    if root_node.leaf:
        root_node.mu[root_node.state] = 0
        root_node.min_val = 0
        print(root_node.state, root_node.mu)
        # YOUR CODE HERE
        # raise NotImplementedError()
    else:
        # recurse (post order traversal)
        for w in root_node.children:
            sankoff_fill(w, states_alphabet)
        for s in alphabet:
            sum = 0;
            for child in root_node.children:
                min_value = math.inf
                for t in alphabet:
                    if(min_value>cost(s,t)+child.mu[t]):
                        min_value=cost(s,t)+child.mu[t]
                sum+=min_value
            root_node.mu[s] = sum
        
        root_node.min_val = min(root_node.mu.values())
        print(root_node.state, root_node.mu, root_node.min_val)
        # post order processing of current node
        # YOUR CODE HERE
        # raise NotImplementedError()
    
def sankoff_backtrace(root_node, state_alphabet):
    """
    Takes a reference to the root node of a character based phylogeny tree and 
    fills in the state values with the appropriate character. 
    
    """
    # base case
    if root_node.leaf:
        return
    
    # process current node
    if root_node.root:
        # YOUR CODE HERE
        root_node.state = min(root_node.mu, key=root_node.mu.get)
        # print(root_node.state)
    else:
        parent = root_node.parent;
        parent_state = parent.state
        min_state=''
        min_val = math.inf
        for s in alphabet:

           val= cost(parent_state,s)+root_node.mu[s]
           if(val<min_val):
              min_val = val
              min_state=s
        # YOUR CODE HERE
        root_node.state= min_state
    
    # recurse
    for w in root_node.children:
        sankoff_backtrace(w, state_alphabet)            

In [14]:
### Student tests (You may add more)

alphabet = ['A', 'C', 'T', 'G']

# Construct tree with nodes named 0-6 from leaves to root
#            6
#          /  \
#         4    5
#       / \   / \
#      0  1  2  3
#      A  C  T  G
#

leaves = []
for i,s in enumerate(['A', 'C', 'T', 'G']):
    leaves.append(Node(i, s, alphabet, root=False, leaf=True))
node1 = Node(4, None,alphabet,False, False)
node1.add_child(leaves[0])
node1.add_child(leaves[1])
node2 = Node(5, None,alphabet,False, False)
node2.add_child(leaves[2])
node2.add_child(leaves[3])
root = Node(6, None, alphabet, True, False)
root.add_child(node1)
root.add_child(node2)

sankoff_fill(root, alphabet)
# print(root.mu.values())
sankoff_backtrace(root,alphabet)

nt.assert_equal(root.min_val, 3)
nt.assert_equal(root.state, 'A')
nt.assert_equal(node1.state, 'A')
nt.assert_equal(node2.state, 'A')

A {'A': 0, 'C': inf, 'T': inf, 'G': inf}
C {'A': inf, 'C': 0, 'T': inf, 'G': inf}
None {'A': 1, 'C': 1, 'T': 2, 'G': 2} 1
T {'A': inf, 'C': inf, 'T': 0, 'G': inf}
G {'A': inf, 'C': inf, 'T': inf, 'G': 0}
None {'A': 2, 'C': 2, 'T': 1, 'G': 1} 1
None {'A': 3, 'C': 3, 'T': 3, 'G': 3} 3


In [15]:
# HIDDEN TEST CELL DO NOT MODIFY


In [16]:
# HIDDEN TEST CELL DO NOT MODIFY
